<a href="https://colab.research.google.com/github/esephaneli/Stylest-V2-AI-Style-Advisor-Multimodal-/blob/main/stylest_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stylest V2 – AI Style Advisor
Bu notebook, kullanıcı fotoğrafı + ürün fotoğrafı üzerinden
stil uyumu ve kombin önerisi üreten
multimodal bir AI sisteminin özet mimarisini gösterir.


In [ ]:
# ===============================
# Stylest V2 - AI Style Advisor
# Multimodal (Image + Text)
# ===============================

# 1️⃣ Gerekli Kütüphaneler
import base64
import json
from pathlib import Path
from typing import List
from pydantic import BaseModel
from openai import OpenAI

client = OpenAI()

# -------------------------------
# 2️⃣ Pydantic Schema (JSON Output)
# -------------------------------

class StyleAdvice(BaseModel):
    user_style_summary: str
    product_summary: str
    verdict: str
    reasons: List[str]
    size_fit_tips: List[str]
    color_match_tips: List[str]
    outfit_suggestions: List[str]
    confidence: float
    preview_prompt: str


# -------------------------------
# 3️⃣ Görseli modele uygun formata çevirme
# -------------------------------

def image_to_base64(path: str) -> str:
    return base64.b64encode(Path(path).read_bytes()).decode()


# -------------------------------
# 4️⃣ Multimodal LLM Çağrısı
# (User Photo + Product Photo + Preferences)
# -------------------------------

def get_style_advice(
    user_image_path: str,
    product_image_path: str,
    preferences: str,
    gender: str
) -> StyleAdvice:

    gender_hint = {
        "Erkek": "male model",
        "Kadın": "female model",
        "Belirtmek istemiyorum": "adult model"
    }.get(gender, "adult model")

    prompt = f"""
    Sen bir AI stil danışmanısın.
    Kullanıcı tercihi: {preferences}
    Preview model cinsiyeti: {gender_hint}

    Görevlerin:
    - Stil uyumunu değerlendir
    - Kombin önerileri üret
    - Karar gerekçelerini açıkla
    - Concept preview için bir prompt yaz
    """

    response = client.responses.create(
        model="gpt-4o-mini",
        input=[{
            "role": "user",
            "content": [
                {"type": "input_text", "text": prompt},
                {"type": "input_image", "image_base64": image_to_base64(user_image_path)},
                {"type": "input_image", "image_base64": image_to_base64(product_image_path)}
            ]
        }],
        text={"format": {"type": "json"}}
    )

    return StyleAdvice.model_validate_json(response.output_text)


# -------------------------------
# 5️⃣ Çıktıyı Uygulama Diline Çevirme
# -------------------------------

def render_user_friendly_output(result: StyleAdvice):
    print(f"Karar: {result.verdict}")
    print(f"Güven Skoru: {result.confidence}\n")

    print("Kombin Önerileri:")
    for item in result.outfit_suggestions:
        print("-", item)


# -------------------------------
# 6️⃣ Örnek Çalıştırma
# -------------------------------

result = get_style_advice(
    user_image_path="user.jpg",
    product_image_path="product.jpg",
    preferences="Sonbahar için rahat ve sade kombin",
    gender="Erkek"
)

render_user_friendly_output(result)
